# Spanish high speed rail system ticket pricing
## Inspiration
The Guru and Team, have in mind a data product that takes advantage of this data to allow users getting better prices when buying train tickets.

We wanted to share the development of this product openly from the very beginning, from data collection to web development. As Data Scientists we understand the importance of getting feedback and ideas from other fellows. 

## Title

    
## Sources
- [Kaggle ?? competition](kaggle.com/thegurus/spanish-high-speed-rail-system-ticket-pricing)
- Citation

![Spain](./data/rail_route.png)

In [ ]:
# importing libraries 
import numpy as np
import pandas as pd
import seaborn as sns
import plotly_express as px
import datetime as dt 

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

## Reading data

# Note for linting the code

```
# lint
conda install flake8
conda install pycodestyle
pip install pycodestyle_magic
```

## In Jupyter
### To load
```
%load_ext pycodestyle_magic
```

### To run
```
%%flake8
```

In [ ]:
# Reading the source dataset, CSV file
# located under the capstone project data folder

# loading renfe data into the DataFrame
df = pd.read_csv(
    './data/renfe.csv',
    parse_dates=['insert_date', 'start_date', 'end_date'],nrows = 250000)
# df = pd.read_csv('..//Capstone Projects/data/renfe.csv')

In [ ]:
# dataset size Rows and Columns
print(df.shape)

In [ ]:
# Feature datatype and number of observations
print(df.info())

In [ ]:
# drop the column 'unnamed' not used
df.drop(columns="Unnamed: 0",inplace = True)

df_final = df[df['price'].isna()]

In [ ]:
# rearranging columns, Price feature is the target variable
cols = ['insert_date', 'origin', 'destination',
        'start_date', 'end_date', 'train_type', 'train_class',
        'fare', 'price']
df = df[cols]

# final prediction dataframe
df_final = df_final[cols]

In [ ]:


#print(df[df['train_class'] == 'Cama G. Clase'].head(1))

#print(df[(df['train_type'] == 'R. EXPRES')].head(1))

cond = df.train_class == 'Cama G. Clase'
rows = df.loc[cond, :]
df_final = df_final.append(rows, ignore_index=True)


cond = df.train_class == 'Cama Turista'
rows = df.loc[cond, :]
df_final = df_final.append(rows, ignore_index=True)

cond = df.train_type == 'R. EXPRES'
rows = df.loc[cond, :]
df_final = df_final.append(rows, ignore_index=True)


cond = df.fare == 'Adulto ida'
rows = df.loc[cond, :]
df_final = df_final.append(rows, ignore_index=True)


print(df.train_class.unique())
print(df_final.train_class.unique())
print("--------------------")
print(df.train_type.unique())
print(df_final.train_type.unique())
print("--------------------")
print(df.fare.unique())
print(df_final.fare.unique())

In [ ]:
# columns containing null values
null_columns = df.columns[df.isnull().any()].tolist()

# missing values count for each columns
print(df[null_columns].isnull().sum())

In [ ]:
# columns containing null values in df_final
null_columns = df_final.columns[df_final.isnull().any()].tolist()

# missing values count for each columns in df_final dataframe used for predict
print(df_final[null_columns].isnull().sum())

In [ ]:
# drop the nan values in df dataframe and see anymore nan values in dataset
df = df.dropna(axis=0)
print(df.isna().sum())

In [ ]:
# drop the nan values in df dataframe and see anymore nan values in dataset
df_final['train_class'].dropna(axis=0)
print(df_final.isna().sum())

In [ ]:
# the values in the price column are modified to remove special chars and space
df.fare.unique()

df_final.head(20)

In [ ]:
# cleaning the data containing spaces and special chars

df['fare'].replace(
    to_replace=['Promo +'],
    value='Promo-Plus',
    inplace=True
)
df['fare'].replace(
    to_replace=['Adulto ida'],
    value='Adulto-ida',
    inplace=True
)

# cleaning the data containing spaces and special chars for final 
# dataframe used for predict
df_final['fare'].replace(
    to_replace=['Promo +'],
    value='Promo-Plus',
    inplace=True
)
df_final['fare'].replace(
    to_replace=['Adulto ida'],
    value='Adulto-ida',
    inplace=True
)

In [ ]:
# the values in the train_type column are modified to remove special chars and space
df.train_type.unique()

In [ ]:
df['train_type'].replace(
    to_replace=['AV City'],
    value='AV-City',
    inplace=True
)
df['train_type'].replace(
    to_replace=['R. EXPRES'],
    value='R-EXPRES',
    inplace=True
)

df_final['train_type'].replace(
    to_replace=['AV City'],
    value='AV-City',
    inplace=True
)
df_final['train_type'].replace(
    to_replace=['R. EXPRES'],
    value='R-EXPRES',
    inplace=True
)

In [ ]:
# the values in the train_class column are modified to remove special chars and space

df['train_class'].replace(
    to_replace=['Turista con enlace'],
    value='Turista-con-enlace',
    inplace=True
)
df['train_class'].replace(
    to_replace=['Turista Plus'],
    value='Turista-Plus',
    inplace=True
)
df['train_class'].replace(
    to_replace=['Cama G. Clase'],
    value='Cama-G-Clase',
    inplace=True
)
df['train_class'].replace(
    to_replace=['Cama Turista'],
    value='Cama-Turista',
    inplace=True
)
df.train_class.unique()

df_final['train_class'].replace(
    to_replace=['Turista con enlace'],
    value='Turista-con-enlace',
    inplace=True
)
df_final['train_class'].replace(
    to_replace=['Turista Plus'],
    value='Turista-Plus',
    inplace=True
)
df_final['train_class'].replace(
    to_replace=['Cama G. Clase'],
    value='Cama-G-Clase',
    inplace=True
)
df_final['train_class'].replace(
    to_replace=['Cama Turista'],
    value='Cama-Turista',
    inplace=True
)

In [ ]:
# Converting date time values into
cols = ['insert_date', 'start_date', 'end_date']
for col in cols:
    col_name = col.split('_')[0]
    df[col_name + '_hour'] = df[col].dt.hour
    df[col_name + '_minute'] = df[col].dt.minute
    df[col_name + '_weekday'] = df[col].dt.dayofweek  # consider class/label instead
    df[col_name + '_day'] = df[col].dt.day
    df[col_name + '_month'] = df[col].dt.month
    df_final[col_name + '_hour'] = df_final[col].dt.hour
    df_final[col_name + '_minute'] = df_final[col].dt.minute
    df_final[col_name + '_weekday'] = df_final[col].dt.dayofweek  # consider class/label instead
    df_final[col_name + '_day'] = df_final[col].dt.day
    df_final[col_name + '_month'] = df_final[col].dt.month
   # df[col_name + '_year'] = df[col].dt.year

In [ ]:
# determining the duration from origin to destination e.g. BARCELONA-MADRID
df['duration_hrs'] = df['end_date'] - df['start_date']
df['duration_hrs'] = df['duration_hrs'] / np.timedelta64(1, 'h')

# determining the duration from origin to destination e.g. BARCELONA-MADRID
df_final['duration_hrs'] = df_final['end_date'] - df_final['start_date']
df_final['duration_hrs'] = df_final['duration_hrs'] / np.timedelta64(1, 'h')

In [ ]:
df['days'] = df['end_day'] - df['start_day']  
df_final['days'] = df_final['end_day'] - df_final['start_day']  

In [ ]:

print(df[df['days']>0])

In [ ]:
# Journey duration info between different routes
df['duration_hrs'].describe()


In [ ]:
# Price info
df['price'].describe()

In [ ]:
# total numbers of the days the observation exists
total_days = max(df['end_date']) - min(df['start_date'])
print("total number days in the observations :", total_days)
print('-----------------------')

In [ ]:
# the new feature route is generated from the orign and destination input features
df['route'] = df.apply(lambda row: 
                       '%s-%s' % (row['origin'], row['destination']),
                       axis=1)
df['route'].value_counts()

# Repeating the steps to keep final dataframe consistent input features
df_final['route'] = df_final.apply(lambda row: 
                       '%s-%s' % (row['origin'], row['destination']),
                       axis=1)

In [ ]:
# list the distinct routes from the source dataset
routes_list = df['route'].unique()
print(routes_list)

In [ ]:
# function definition to display the min and max date range 
def minmaxdate(df, col):
    return(df[col].min(), df[col].max())

# display the min and max date range for date columns
print(minmaxdate(df, 'insert_date'))
print(minmaxdate(df, 'start_date'))
print(minmaxdate(df, 'end_date'))
print('-----------------------')

In [ ]:
agg_ = {'price': ['min', 'max']}
df[['route', 'price']].groupby('route').agg(agg_)

In [ ]:
# finding min and max price for each of routes
print('Route                 Min   Max')
for r in routes_list:
    print('%-20s %.1f %.1f' % (
        r,
        min((df[df['route'] == r]).price),
        max((df[df['route'] == r]).price)))

In [ ]:
df['start_date_only'] = df['start_date'].dt.date
# print(sd_list_df.route.value_counts())

# repeating the step to create new feature start_date_only for final dataframe
df_final['start_date_only'] = df_final['start_date'].dt.date

In [ ]:
sns.distplot(df['price']);

In [ ]:
# anomalies of longer duration it happened on weekends
#px.scatter(df, x="duration_hrs", y="start_weekday", color="route")

In [ ]:
sns.boxplot(y='price', data=df);

In [ ]:
 sns.scatterplot(x='price',y='route',data=df)

In [ ]:
px.scatter(df, x="duration_hrs", y="train_class", color="price")

In [ ]:
px.scatter(df, x="route", y="price", color="fare")

In [ ]:
px.scatter(df, x="origin", y="destination", color="price")
df_final.columns

In [ ]:
feature_columns = [
    'origin', 'destination', 'train_type', 'train_class',
    'fare', 'route', 'insert_hour', 'insert_minute',
    'insert_weekday', 'insert_day', 'insert_month',
    'insert_year', 'start_hour', 'start_minute',
    'start_weekday', 'start_day', 'start_month',
    'start_year', 'end_hour', 'end_minute',
    'end_weekday', 'end_day', 'end_month', 'end_year']

df = pd.get_dummies(df, columns=[
    'origin', 'destination', 'train_type','train_class',
    'fare', 'route', 'start_weekday',
    ])

In [ ]:
df_final = pd.get_dummies(df_final, columns=[
    'origin', 'destination', 'train_type', 'train_class',
    'fare', 'route', 'start_weekday',
    ])

In [ ]:
df = df.drop(columns=['insert_date', 'start_date', 'end_date'])


# repeat the unused feature after the date value trnasformation into granual level
df_final = df_final.drop(columns=['insert_date', 'start_date', 'end_date'])


In [ ]:
df = df.drop(columns=['start_date_only','end_weekday','insert_weekday'], axis=1)
# print(df.info())

df_final = df_final.drop(columns=['start_date_only','end_weekday','insert_weekday'], axis=1)
# print(df.info())

# define the target variable (dependent variable) as y
y = df['price']

# define the target variable (dependent variable) as y for y pred
y_pred = df_final['price']

include_cols = [c for c in df.columns if c != 'price']

include_Xpredcols = [c for c in df_final.columns if c != 'price']

X = df[include_cols]

X_pred = df_final[include_Xpredcols]



In [ ]:
print(include_cols)
df_final.columns

In [ ]:
# Create training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=16000)

In [ ]:
# Flag intermediate output

show_steps = True   # for testing/debugging
show_steps = False  # without showing steps
print(X_test.shape[0])

In [ ]:
# Use Forward Feature Selection to pick a good model

# start with no input variables
included = []
r2_list = []
adjusted_r2_list = []
model = LinearRegression()
n = X_test.shape[0]

for _ in range(X.shape[1]):
    # keep track of model and parameters
    best = ('', 0, 0)
    # list the input variables to be evaluated
    excluded = list(set(X.columns) - set(included))
    # for each remaining feature to be evaluated
    for new_column in excluded:
        # fit the model with the Training data
        fit = model.fit(X_train[included + [new_column]], y_train)
        # calculate the score (R^2 for Regression)
        r2 = fit.score(X_train[included + [new_column]], y_train)
        # number of features in this model
        k = len(included) + 1
        # calculate the adjusted R^2
        adjusted_r2 = 1 - (((1 - r2) * (n - 1)) / (n - k - 1))

        # if model improves
        if adjusted_r2 > best[2]:
            # record new parameters
            best = (new_column, r2, adjusted_r2)
    # END for new_column in excluded

    r2_list.append(best[1])
    adjusted_r2_list.append(best[2])

    included.append(best[0])
    excluded = list(set(excluded) - set(best[0]))
    print('Add %-30s with R^2 = %.4f and adjusted R^2 = %.4f' % (best))

print('')
print('Resulting features:')
print(', '.join(included))

In [ ]:
# length of the included feature list
type(included)
len(included)

In [ ]:
# Chart both R^2 and Adjusted R^2

# define chart size
plt.figure(figsize=(10, 5))
# plot each metric 
plt.plot(range(0, len(included)), r2_list, label='$R^2$')
plt.plot(range(0, len(included)), adjusted_r2_list, label='$Adjusted \: R^2$')
# add some better visualisation
plt.xlabel('Number of Features')
plt.legend()
# output the chart
plt.show()

In [ ]:
cols_A = df.columns[:38]
cols_B = df.columns[38:]

In [ ]:
## Calculate Ridge Regression model

# create a model object to hold the modelling parameters
clf = Ridge()

# keep track of the intermediate results for coefficients and errors
coefs = []
errors = []

# create a range of alphas to calculate
alphas = np.logspace(-6, 6, 200)

# Train the model with different regularisation strengths
for a in alphas:
    clf.set_params(alpha = a)
    clf.fit(X, y)
    coefs.append(clf.coef_)
    errors.append(mean_squared_error(clf.coef_, fit.coef_))

In [ ]:
# Display results
plt.figure(figsize = (20, 6))

plt.subplot(121)
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title('Ridge coefficients as a function of the regularisation')
plt.axis('tight')

plt.subplot(122)
ax = plt.gca()
ax.plot(alphas, errors)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('error')
plt.title('Coefficient error as a function of the regularisation')
plt.axis('tight')

plt.show()

In [ ]:
df[included[:33]].dtypes

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

colormap = plt.cm.RdBu
plt.figure(figsize=(10, 10))
plt.title('Pearson Correlation of Features', size=15)
sns.heatmap(df[included[:32]].corr(),
            linewidths=0.1,
            vmax=1.0,
            square=True,
            cmap=colormap,
            linecolor='white',
            annot=False)
plt.show()

X = df.iloc[:,0:64]  #independent columns
y = df.iloc[:,-1]    #target column i.e price range
#get correlations of each features in dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(200,200))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
print(X_train.head(1))

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)


In [ ]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Train Score:", train_score)
print("Test Score:", test_score)

In [ ]:
print(X_pred.info())
y_pred = model.predict(X_pred)


In [ ]:
df_final['y_hats'] = y_pred 
df_out = pd.merge(df_final,df_final[['y_hats']],how = 'left',left_index = True, right_index = True)

In [ ]:
print(df_out.head(10))

In [ ]:
df_out.describe()

df_out.info()

In [ ]:
df.describe()